# **Aprendizaje Automático - Regresión**

- Francisco Prados Abad
- Paola León Tarife
- Julia de Enciso García
- Paula Samper López
- Camino Rodríguez Pérez-Carral
- Lucía Yan Wu


****

En este notebook hemos utilizado distintos modelos para resolver la tarea de regresión, donde tenemos que predecir el valor de la variable *ScoreRiesgo*.

En primer lugar, fijamos la semilla y hacemos los imports necesarios.

In [ ]:
import torch
import random
import numpy as np
import keras

# Set seed
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
np.random.seed(42)
random.seed(42)
keras.utils.set_random_seed(42)

In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.metrics import mean_absolute_error, r2_score, accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor, StackingRegressor, BaggingRegressor
from sklearn.neural_network import MLPRegressor
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasRegressor

### Cargar los datos

Ahora cargamos los datos que ya han sido preprocesados.

In [ ]:
# Accedemos a las bases de datos procesadas
X_train = pd.read_csv('data/regresión/X_train.csv')
y_train = pd.read_csv('data/regresión/y_train.csv')
X_test = pd.read_csv('data/regresión/X_test.csv')
y_test = pd.read_csv('data/regresión/y_test.csv')

# Tarea de Regresión

A continuación pasamos a explicar los modelos que hemos probado. Hemos probado modelos simples, como la regresión lineal y un árbol de decisión. También hemos utilizado redes de neuronas, y hemos montado distintos ensembles para intentar mejorar nuestros resultados.

Ajustaremos los hiperparámetros de los modelos mediante RandomizedSearchCV y GridSearch CV.

El error que hemos utilizado para comparar los resultados de los distintos modelos es el Mean Absolute Error (MAE).

### Linear Regression

El primer modelo que probamos fue la regresión lineal, que es un modelo bastante simple. Hemos ajustado el parámetro *n_jobs* mediante GridSearchCV, que define el grado de paralelismo utilizado.

In [ ]:
# Definimos el modelo
lr = LinearRegression()

# Definir distintos valores de los parámetros
param_grid_lr = {'n_jobs': [0, 50, 100, 200]}

rand_lr = GridSearchCV(estimator=lr, param_grid=param_grid_lr, scoring='neg_mean_absolute_error', cv=5, verbose=False)
rand_lr.fit(X_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=LinearRegression(),
             param_grid={'n_jobs': [0, 50, 100, 200]},
             scoring='neg_mean_absolute_error', verbose=False)

In [ ]:
rand_lr.best_score_

-3.236717540371663

In [ ]:
rand_lr.best_params_

{'n_jobs': 0}

In [ ]:
rand_lr.cv_results_

{'mean_fit_time': array([0.19501157, 0.13257389, 0.05515728, 0.069416  ]),
 'std_fit_time': array([0.06055531, 0.04352395, 0.01287706, 0.01999354]),
 'mean_score_time': array([0.00697765, 0.00766096, 0.00558639, 0.00429897]),
 'std_score_time': array([0.00325945, 0.00351325, 0.00248819, 0.00238992]),
 'param_n_jobs': masked_array(data=[0, 50, 100, 200],
              mask=[False, False, False, False],
        fill_value=999999),
 'params': [{'n_jobs': 0}, {'n_jobs': 50}, {'n_jobs': 100}, {'n_jobs': 200}],
 'split0_test_score': array([-3.24093687, -3.24093687, -3.24093687, -3.24093687]),
 'split1_test_score': array([-3.22058881, -3.22058881, -3.22058881, -3.22058881]),
 'split2_test_score': array([-3.2172527, -3.2172527, -3.2172527, -3.2172527]),
 'split3_test_score': array([-3.26204418, -3.26204418, -3.26204418, -3.26204418]),
 'split4_test_score': array([-3.24276516, -3.24276516, -3.24276516, -3.24276516]),
 'mean_test_score': array([-3.23671754, -3.23671754, -3.23671754, -3.23671754]

#### Best LR

Ahora, probamos el mejor modelo en los datos de test.

In [ ]:
# Definimos el modelo
lr = LinearRegression(n_jobs=0)

# Entrenamos nuestro modelo
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

# MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error for y_pred: {mae}")


Mean Absolute Error for y_pred: 3.2613459427699567


El MAE obtenido es bastante alto, de 3.26. Esto nos indica que la relación que existe entre *ScoreRiesgo* y el resto de atributos no es una relación lineal.

### Árbol de decision para regresión

El siguiente modelo que hemos probado es un árbol de decisión.

Hemos ajustado los siguientes hiperparámetros mediante *RandomizedSearchCV*:


*   **max_depth**: profundidad máxima del árbol de decisión.
*   **min_samples_leaf**: número mínimo de datos que hacen falta para formar una hoja.
*   **max_features**: porcentaje de los atributos que se tienen en cuenta para dividir un nodo.
*   **min_samples_split**: número de datos que hacen falta para dividir un nodo.

In [ ]:
dt = DecisionTreeRegressor(random_state=42)

# Definir distintos valores de los parámetros
param_grid_rf = {'max_depth' : [4, 8, 12, None],
               'min_samples_leaf': [1, 3, 5],
               'max_features': [0.5, 1.0],
               'min_samples_split': [2, 5]}

# Busqueda de hiperparámetros
rand_DT = RandomizedSearchCV(estimator=dt, param_distributions=param_grid_rf, scoring='neg_mean_absolute_error', cv=5, verbose=False)
rand_DT.fit(X_train, y_train.values.ravel())

# resultados
best_score = rand_DT.best_score_
best_params = rand_DT.best_params_

print(best_score)
print(best_params)

-2.527342292965824
{'min_samples_split': 5, 'min_samples_leaf': 5, 'max_features': 0.5, 'max_depth': 12}


#### Best DT

Tras ajustar los hiperparámetros, entrenamos el modelo.

In [ ]:
dt = DecisionTreeRegressor(random_state=42, max_features=0.5, min_samples_split=5, min_samples_leaf=5, max_depth=12)

dt.fit(X_train, y_train.values.ravel())

y_pred = dt.predict(X_test)

# MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error for y_pred: {mae}")

Mean Absolute Error for y_pred: 2.5256624217676213


El MAE que hemos obtenido en los datos de test es de 2.52. Hemos mejorado el error respecto al modelo de regresión lineal, pero sigue siendo necesario probar otros modelos que capten mejor las relaciones entre los distintos atributos.

### Red de Neuronas (Keras)

A continuación, vamos a probar una red de neuronas de *Keras* sencilla de dos capas.

Hemos ajustado los hiperparámetros mediante *KerasRegressor*, que permite utilizar *RandomizedSearchCV* con la red aunque no sea un modelo de *sci-kit learn*. Los hiperparámetros que hemos ajustado son:


*   **batch_size:** tamaño del batch.
*   **epochs:** número de epochs para entrenar el modelo.
*   **model__neurons_layer1:** número de neuronas en la primera capa oculta de la red.
*   **model__neurons_layer2:** número de neuronas en la segunda capa oculta de la red.
*   **model__optimizer:** optimizador del modelo para el ajuste de parámetros en el entrenamiento.


In [ ]:
# Función que construye el modelo
def build_model(neurons_layer1=64, neurons_layer2=32, optimizer='adam'):
    model = keras.Sequential([
        layers.Input(shape=(X_train.shape[1],)),  # Capa de entrada
        layers.Dense(neurons_layer1, activation='relu'),  # Primera capa oculta
        layers.Dense(neurons_layer2, activation='relu'),  # Segunda capa oculta
        layers.Dense(1)  # Capa de salida
    ])
    model.compile(optimizer=optimizer, loss='mean_absolute_error')  # Compilar el modelo
    return model

# Envolver el modelo en KerasRegressor
model = KerasRegressor(model=build_model, verbose=0)

# Definir los parámetros para la búsqueda en cuadrícula
param_grid = {
    'batch_size': [16, 32, 64],         # Probar diferentes tamaños de batch
    'epochs': [50, 100, 200],           # Diferentes épocas
    'model__neurons_layer1': [32, 64, 128],  # Cantidad de neuronas en la primera capa
    'model__neurons_layer2': [16, 32, 64],   # Cantidad de neuronas en la segunda capa
    'model__optimizer': ['adam', 'rmsprop']  # Diferentes optimizadores
}

# Configurar el GridSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                                   n_iter=5, cv=3, scoring='neg_mean_absolute_error', verbose=1)

# Ejecutar la búsqueda
random_search_result = random_search.fit(X_train, y_train)

# Mejor resultado
print(f"Best MAE: {-random_search_result.best_score_}")
print(f"Best params: {random_search_result.best_params_}")

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Best MAE: 1.6773001161649308
Best params: {'model__optimizer': 'adam', 'model__neurons_layer2': 32, 'model__neurons_layer1': 128, 'epochs': 200, 'batch_size': 64}


Tras obtener los hiperparámetros, entrenamos el modelo. Podemos ver que el MAE obtenido en el conjunto de test es de 1.654, una mejora considerable respecto a los modelos anteriores.

In [ ]:
# Función para crear el modelo con los mejores hiperparámetros
def build_best_model():
    model = models.Sequential([
        layers.Input(shape=(X_train.shape[1],)),  # Capa de entrada
        layers.Dense(128, activation='relu'),  # Capa oculta 1 con 128 neuronas
        layers.Dense(32, activation='relu'),   # Capa oculta 2 con 32 neuronas
        layers.Dense(1)  # Capa de salida
    ])

    # Compilar el modelo usando el optimizador 'adam'
    model.compile(optimizer=Adam(), loss='mean_absolute_error')

    return model

# Crear el modelo usando los mejores hiperparámetros
best_model = build_best_model()

# Entrenar el modelo con batch_size=64 y epochs=200
best_model.fit(X_train, y_train, epochs=200, batch_size=64, verbose=1)

# Hacer predicciones en el conjunto de prueba
y_pred_best = best_model.predict(X_test)

# Evaluar el modelo con MAE
mae_best = mean_absolute_error(y_test_r, y_pred_best)
print(f"Mean Absolute Error on test set: {mae_best}")


Epoch 1/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 21.6568
Epoch 2/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3.3619
Epoch 3/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 2.7172
Epoch 4/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.2510
Epoch 5/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 2.0757
Epoch 6/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.9721
Epoch 7/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.9159
Epoch 8/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1.9035
Epoch 9/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.8710
Epoch 10/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.8390
Epoch 11/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.8280
Epoch 12/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.8127
Epoch 13/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1.7974
Epoch 14/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.7849
Epoch 15/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 

#### Best Keras Model

Sin embargo, el mejor modelo lo hemos obtenido en una de las muchas pruebas que hemos a mano. La arquitectura de la red es la siguiente:

*   100 epochs
*   Batch de 32
*   Primera capa oculta con 64 neuronas
*   Segunda capa oculta con 32 neuronas
*   Optimizador Adam

Este modelo nos ha proporcionado el mejor MAE sobre los datos de test, de 1.628.

In [ ]:
# Define the Neural Network model
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # Input layer
    layers.Dense(64, activation='relu'),  # Hidden layer
    layers.Dense(32, activation='relu'),  # Another hidden layer
    layers.Dense(1)  # Output layer
])
# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)  # Adjust epochs and batch_size as needed

# Make predictions
y_pred = model.predict(X_test)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error for y_pred: {mae}")

Epoch 1/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 17.2734
Epoch 2/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 2.9868
Epoch 3/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2.4655
Epoch 4/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 2.2493
Epoch 5/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 2.1062
Epoch 6/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 2.0530
Epoch 7/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.9886
Epoch 8/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.9438
Epoch 9/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 1.8962
Epoch 10/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 1.8529
Epoch 11/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 1.8336
Epoch 12/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.8185
Epoch 13/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.8122
Epoch 14/100
1242/1242 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - loss: 1.7890
Epoch 15/100
1

Cabe destacar quee intentamos añadir capas de dropout en la red, pero empeoraban el error. Probablemente se deba a que estamos tratando con una red muy pequeña.

### Red de Neuronas (MLPRegressor)

A continuación, probamos otra red de neuronas, en este caso un *MLPRegressor*. Esta red tendrá una única capa oculta.
Ajustamos los siguientes hiperparámetros mediante *RandomizedSearchCV*:



*   **hidden_layer_sizes:** número de neuronas en la capa oculta
*   **solver:** optimizador del modelo para el ajuste de parámetros en el entrenamiento.
*   **alpha:** valor del hiperparámetro de la regularización L2, una técnica para reducir el overfitting.
*   **max_iter:** número de epochs.

In [ ]:
MLP = MLPRegressor()

# Definir distintos valores de los parámetros
param_grid_MLP = {'hidden_layer_sizes': [50, 100, 200],
                  'solver': ['sgd', 'adam'],
                  'alpha': np.arange(0, 1, 0.05),
                  'max_iter': [500, 1000]}

rand_MLP = RandomizedSearchCV(estimator=MLP, param_distributions=param_grid_MLP, n_iter=5, scoring='neg_mean_absolute_error', cv=5, verbose=False)
# rand_MLP = GridSearchCV(estimator=MLP, param_grid=param_grid_MLP, scoring='neg_mean_absolute_error', cv=5, verbose=False)
rand_MLP.fit(X_train, y_train.values.ravel())

print('Best Score: ', rand_MLP.best_score_)
print('Best Parameters :', rand_MLP.best_params_)
print('CV Results: ', rand_MLP.cv_results_)

Best Score:  -1.8238276780890612
Best Parameters : {'solver': 'adam', 'max_iter': 500, 'hidden_layer_sizes': 200, 'alpha': 0.35000000000000003}
CV Results:  {'mean_fit_time': array([17.58362207, 40.61703763, 61.59212222, 43.13875132, 55.59276347]), 'std_fit_time': array([ 4.24540745, 21.29543879,  8.91683407,  4.522613  , 20.56658598]), 'mean_score_time': array([0.00483322, 0.00599322, 0.01118193, 0.00563693, 0.01095567]), 'std_score_time': array([0.00155386, 0.0001756 , 0.00103705, 0.00202193, 0.00116125]), 'param_solver': masked_array(data=['sgd', 'sgd', 'adam', 'adam', 'sgd'],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_max_iter': masked_array(data=[500, 1000, 500, 500, 500],
             mask=[False, False, False, False, False],
       fill_value=999999), 'param_hidden_layer_sizes': masked_array(data=[50, 100, 200, 50, 200],
             mask=[False, False, False, False, False],
       fill_value=999999), 'param_al

#### Best MLPRegressor

Entrenamos el modelo con los hiperparámetros que hemos encontrado y lo probamos sobre los datos de test.
En este caso al final nos dimos cuenta de que obteniamos mejores resultados con el solver 'adam'. Esto seguramente es debido a que n_iter era bajo y no llegó a probar esta combinación


In [ ]:
MLP = MLPRegressor(hidden_layer_sizes=(200,),
                   max_iter=1000,  # Increase the max iterations
                   solver='sgd',  # Optimizer
                   learning_rate_init=0.001,  # Adjust learning rate
                   random_state=42)

MLP.fit(X_train, y_train.values.ravel())

# Hacer predicciones con los datos de prueba
y_pred = MLP.predict(X_test)

# MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error for y_pred: {mae}")

Mean Absolute Error for y_pred: 1.8166929505250877


Vemos que el MAE es de 1.817. Sigue siendo un error menor que el del árbol de decisión pero es peor que el de la red de *Keras*. Probablemente esto se deba a que la arquitectura de nuestras dos redes es distinta, y también es posible que al estar implementadas en librerías diferentes, existan algunos detalles en las implementaciones internas que no sean iguales.

### Random Forest

Ahora vamos a pasar a probar distintos ensembles para intentar mejorar los distintos resultados que hemos obtenido. Dentro de los ensembles homogéneos, que usan como base el mismo tipo de modelo, el primero que probaremos será el *Random Forest*, que usa árboles de decisión como base.

Hemos ajustado los siguientes hiperparámetros mediante *GridSearchCV*:


*   **n_estimators:** número de árboles de decisión
*   **max_features**: proporción de los atributos que se tienen en cuenta para dividir un nodo.
*   **min_samples_split**: número de datos que hacen falta para dividir un nodo.



In [ ]:
rf = RandomForestRegressor(random_state=42)

# Definir distintos valores de los parámetros
param_grid_rf = {'n_estimators': [50, 100, 200],
               'max_features': [0.5, 1.0],
               'min_samples_split': [2,3,5]}

# rand_RF = RandomizedSearchCV(estimator=rf, param_distributions=param_grid_rf, n_iter=5, scoring='neg_mean_absolute_error', cv=5, verbose=False)
rand_RF = GridSearchCV(estimator=rf, param_grid=param_grid_rf, scoring='neg_mean_absolute_error', cv=5, verbose=False)
rand_RF.fit(X_train, y_train.values.ravel())

# resultados
best_score = rand_RF.best_score_
best_params = rand_RF.best_params_

print(best_score)
print(best_params)

-1.9988038188342607
{'max_features': 1.0, 'min_samples_split': 3, 'n_estimators': 200}


#### Best RF

Ahora entrenamos el mejor modelo.

In [ ]:
rf = RandomForestRegressor(random_state=42, max_features=1.0, min_samples_split=3, n_estimators=200)

rf.fit(X_train, y_train.values.ravel())

y_pred = rf.predict(X_test)

# MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error for y_pred: {mae}")

Mean Absolute Error for y_pred: 2.0338967036985207


Hemos obtenido un MAE de 2.033 sobre los datos de test. Las redes de neuronas siguen dando mejores resultados, pero hemos conseguido mejorar el error del árbol de decisión.

### Gradient Boosting

Vale siguiendo con los ensembles tambien probamos el Gradient Boosting regressor, que como hemos visto en clase, los métodos de boosting entrenan modelos de forma secuencial.

Gradient boosting entrena una serie de árboles de decisión.

A traves de una randomized search de los parámetros vimos que lo mejor era usar un número elevado de optimizadores (800), además de otros parámetros, obteniendo así un error medio absoluto de 2.07

In [ ]:
GB = GradientBoostingRegressor(random_state=42)

# Definir distintos valores de los parámetros
param_grid_gb = {'n_estimators': [200, 400, 600, 800],
               'max_features': [0.5, 1.0],
               'min_samples_split': [2, 5]}

# Busqueda de hiperparámetros
rand_GB = RandomizedSearchCV(estimator=GB, param_distributions=param_grid_gb, scoring='neg_mean_absolute_error', cv=5, verbose=False)
rand_GB.fit(X_train, y_train.values.ravel())

# resultados
best_score = rand_GB.best_score_
best_params = rand_GB.best_params_

print(best_score)
print(best_params)

-2.056489326724912
{'n_estimators': 800, 'min_samples_split': 5, 'max_features': 1.0}


#### Best GB

In [ ]:
GB = GradientBoostingRegressor(random_state=42, max_features=1.0, min_samples_split=5, n_estimators=800)

GB.fit(X_train, y_train.values.ravel())

y_pred = GB.predict(X_test)

# MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error for y_pred: {mae}")

Mean Absolute Error for y_pred: 2.078713938637538


### AdaBoost

Luego probamos el AdaBoost para regresión. Este se enfoca en los errores de predicción de los modelos anteriores y asigna más peso a las observaciones que los modelos iniciales no predijeron bien, a las más difíciles de predecir.

Lo probamos utilizando diferentes modelos como estimadores pero obtuvimos los mejores resultados con la red MLP regressor, obteniendo un MAE de 2,27




In [ ]:
# AdaBoost wrapper
ada_mlp = AdaBoostRegressor(estimator=MLPRegressor(hidden_layer_sizes=(200,),
                   max_iter=1000,  # Increase the max iterations
                   solver='sgd',  # Optimizer
                   learning_rate_init=0.001,  # Adjust learning rate
                   random_state=42), n_estimators=50, random_state=42)

# Train the AdaBoost model
ada_mlp.fit(X_train, y_train.values.ravel())

# Make predictions
y_pred_adaMLP = ada_mlp.predict(X_test)

# Evaluate the performance
mae = mean_absolute_error(y_test, y_pred_adaMLP)
print(f"Mean Absolute Error: {mae}")


Mean Absolute Error: 2.270971365854697


### Bagging

También probamos con el Bagging Regressor, que crea múltiples versiones de un modelo (en este caso, una Red Neuronal MLP) y promedia sus predicciones.

Al igual que con el AdaBoost, probamos utilizando como estimadores árboles de decisión, random forest... pero lo que funcionó mejor fue la red MLP, que nos dio un error de 1,77.

Por ahora el mejor resultado junto con la red de Keras.


In [ ]:
# Instantiate the Bagging Regressor
bagging_regressor = BaggingRegressor(
    estimator=MLPRegressor(hidden_layer_sizes=(200,),
                   max_iter=1000,  # Increase the max iterations
                   solver='sgd',  # Optimizer
                   learning_rate_init=0.001,  # Adjust learning rate
                   random_state=42),  # Default is DecisionTreeRegressor
    n_estimators=50,      # Number of base estimators in the ensemble
    random_state=42       # Ensures reproducibility
)

# Fit the model to the training data
bagging_regressor.fit(X_train, y_train.values.ravel())

# Make predictions on the test data
y_pred_bagging = bagging_regressor.predict(X_test)

# Calculate Mean Absolute Error (MAE) for the Bagging Regressor
mae_bagging = mean_absolute_error(y_test_r, y_pred_bagging)
print(f"Mean Absolute Error for Bagging Regressor: {mae_bagging}")


Mean Absolute Error for Bagging Regressor: 1.7762797388367317


### Voting Regressor

También probamos ensembles heterogéneos que mezclan diferentes tipos de modelos, combinando sus predicciones: Voting y Stacking Regressors

Para ambos metimos como estimadores modelos que ya habíamos entrenado: (lr', linear regression) , ('dt', arbol de decision), ('rf', random forest) y ('MLP', Red MLP)


In [ ]:
# Training classifiers
voting = VotingRegressor(estimators=[('lr', lr), ('dt', arbol_decision), ('rf', rf), ('MLP', MLP)])
voting = voting.fit(X_train, y_train.values.ravel())
# Hacer predicciones con los datos de prueba
y_pred = MLP.predict(X_test)

# MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error for y_pred: {mae}")

Mean Absolute Error for y_pred: 1.8166929505250877


### Stacking Regressor

En ambos el MAE que hemos obtenido es el mismo que en la MLP por si sola.

Lo que pensamos que indica que la MLP es significativamente más fuerte que los otros modelos y termina dominando en las predicciones combinadas.


In [ ]:
# Training classifiers
stacking = StackingRegressor(estimators=[('lr', lr), ('dt', arbol_decision), ('rf', rf), ('MLP', MLP)], final_estimator=RidgeCV())
stacking = stacking.fit(X_train, y_train.values.ravel())
# Hacer predicciones con los datos de prueba
y_pred = MLP.predict(X_test)

# MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error for y_pred: {mae}")

Mean Absolute Error for y_pred: 1.8166929505250877


Como conclusiones del trabajo de regresión:

- El mejor resultado lo obtuvimos con la Red nauronal de Keras.

- Los modelos de ensembles ofrecen buenos resultados pero no superan a la red, lo que demuestra que el patrón no lineal de los datos se captura mejor con la red que con otros modelos.

- Para todos los modelos la optimización de hiperparámetros mejoró el rendimiento.

- Los ensembles funcionan mejor cuando los modelos base aportan diversidad. Si un solo modelo como el MLP es dominante, las mejoras no son significativas.
